# ETL - Columbus City Parking Violations and Ticket Status 2013-2018

In [1]:
# Activate your python environment

# Make sure to install this before running:
# pip install pandas
# pip install sqlalchemy
# pip install psycopg2
# pip install datetime

import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

#### Read 1 of 3 CSV Files

In [2]:
# https://discovery.smartcolumbusos.com/dataset/conduent/160c98a1_ad56_4658_8553_5ee8e7d0d953
# Download this file from google drive link in readme as this is a big file
data_csv = "Parking_data.csv"
data_df = pd.read_csv(data_csv)
data_df.head()

C:\Users\Saiteja\Anaconda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,21,25,43,78,91,97) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,nix dt,hrg offcr,dispo,badge,susp til,mail1 dt,iss dt,ent due,viol,ipp no,...,overpd,pay src,more mail,more corr,mail3 pdt,ipp stat,batch,more pays,seiz ind,more hear
0,0,NaN,0,17,0,0,2015205,$0.00,29,0,...,$0.00,7,NaN,NaN,0,NaN,201,NaN,NaN,NaN
1,0,NaN,0,16,0,2015076,2015044,$0.00,14,0,...,$0.00,7,NaN,NaN,0,NaN,201,NaN,NaN,NaN
2,0,NaN,0,13,2016169,2016180,2016154,$0.00,72,0,...,$0.00,7,NaN,NaN,0,NaN,201,NaN,NaN,NaN
3,0,NaN,0,8,0,2015083,2015055,$0.00,29,0,...,$0.00,1,NaN,NaN,0,NaN,201,NaN,NaN,NaN
4,0,NaN,0,10,0,2015342,2015311,$0.00,50,0,...,$0.00,7,NaN,NaN,0,NaN,201,NaN,NaN,NaN


#### Read 2 of 3 CSV Files

In [3]:
#http://opendata.columbus.gov/datasets/parking-meters/data
meters_csv = "Parking_Meters.csv"
meters_df = pd.read_csv(meters_csv)
meters_df.head()

,X,Y,OBJECTID,METER_ID,LOCATION,SIDE_OF_STREET,BLOCKFACE,METER_STATUS,TOW_AWAY_HOURS,METER_TIME,HANDICAP,HOURS_OPERATION,IN_SERVICE,VALET_HOURS,RATE,FOOD_SERVICE_HOURS,TAXI_ZONE_HOURS,CHARGING_STATION,CHARGING_STATION_STATUS
0,1.828631e+06,710272.804713,6941,B314,SYCAMORE ST W,North,HIGH ST - WALL ST,Operational,0,180.0,0.0,8A-10P,14.0,NaN,$.75,NaN,NaN,No,NaN
1,1.828650e+06,710276.813891,6940,B316,SYCAMORE ST W,North,HIGH ST - WALL ST,Operational,0,180.0,0.0,8A-10P,14.0,NaN,$.75,NaN,NaN,No,NaN
2,1.828671e+06,710279.104897,6939,B318,SYCAMORE ST W,North,HIGH ST - WALL ST,Operational,0,180.0,0.0,8A-10P,14.0,NaN,$.75,NaN,NaN,No,NaN
3,1.828692e+06,710281.968408,6938,B320,SYCAMORE ST W,North,HIGH ST - WALL ST,Operational,0,180.0,0.0,8A-10P,14.0,NaN,$.75,NaN,NaN,No,NaN
4,1.828713e+06,710284.832248,6937,B322,SYCAMORE ST W,North,HIGH ST - WALL ST,Operational,0,180.0,0.0,8A-10P,14.0,NaN,$.75,NaN,NaN,No,NaN


#### Read 3 of 3 CSV Files

In [4]:
#The metadata file to decode the column names
#https://data.world/smartcolumbusos/040b5929-db26-4453-920a-ceb282c4359f/workspace/file?filename=geocoded-parking-violations-csv-5.csv
columns_csv = "metadata.csv"
columns_df = pd.read_csv(columns_csv, header=0, encoding = 'unicode_escape')
columns_df.head()

,Data Field,Description
0,ENTITY,Unique system-generated number to identify li...
1,TERMDT,If a license plate was on a fleet and termina...
2,MULTNO,"If a license plate was on a fleet, that fleet..."
3,ENT DUE,Total amount due on the entity
4,ENT OVP,Overpaid amount on the entity


In [5]:
# Rename viol to violation code - was getting a KeyError: "['viol'] not in index" in the next step
data_df = data_df.rename(columns={"viol": "violation_code"})

#### Select columns from both the datasets

In [6]:
# Select required columns from parking_data
data_df = data_df[['ticket','iss dt','fine','violation_code','entity','make','iss time','lat','long','hold ct','badge','pay amt','location','meter']]

# Rename columns 
data_df = data_df.rename(columns={"ticket": "ticket_id",
                                "iss dt": "issue_date", 
                                  "fine": "fine",
                                 "violation_code": "violation_code", 
                                  "entity": "entity",
                                 "make": "car_make", 
                                  "iss time": "issue_time",
                                 "lat": "latitude", 
                                  "long": "longitude",
                                 "hold ct": "total_tickets", 
                                  "badge": "officer_badge",
                                 "pay amt": "amount_paid",
                                 "location": "location",
                                 "meter": "meter_id"})
data_df

,ticket_id,issue_date,fine,violation_code,entity,car_make,issue_time,latitude,longitude,total_tickets,officer_badge,amount_paid,location,meter_id
0,5565039450,2015205,$30.00,29,3487595,DODG,2035,0,0,0,17,$30.00,GREENWOOD LOT,F48
1,5564345326,2015044,$55.00,14,3453663,CHEV,1110,0,0,0,16,$90.00,480 HIGH ST S,NaN
2,5566101825,2016154,$50.00,72,3376679,TOYT,2049,0,0,0,13,$65.00,1249 DENNISON AVE,NaN
3,5564388332,2015055,$30.00,29,3403466,HOND,1010,0,0,0,8,$45.00,BROAD ST W,WN11
4,5565398944,2015311,$55.00,50,3404432,MERC,936,0,0,0,10,$70.00,219 W1ST,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745816,5568123532,2018020,$30.00,29,3673844,VOLK,2052,39965441,82992614,0,19,$0.00,E/S 6TH ST N,NS130
745817,5568123565,2018020,$30.00,29,3673850,TOYT,2101,39964261,82995553,0,19,$30.00,S/S GAY ST E,EP203
745818,5596117556,2018013,$25.00,20,3673719,HYUN,450,0,0,0,2919,$0.00,1611 CUNARD,NaN
745819,5596768910,2018014,$30.00,15,3673734,TOYT,2345,0,0,0,2234,$45.00,I71 SB / E BROAD ST,NaN


In [7]:
# Select required columns from parking_meter
# Rename columns - easier to merge later
meters_df = meters_df.rename(columns={"METER_ID": "meter_id","METER_STATUS": "meter_status","RATE": "rate" })
meters_df.drop(meters_df.columns[[0,1,2,4,5,6,8,9,10,11,12,13,15,16,17,18]], axis = 1, inplace = True) 
meters_df.head()

,meter_id,meter_status,rate
0,B314,Operational,$.75
1,B316,Operational,$.75
2,B318,Operational,$.75
3,B320,Operational,$.75
4,B322,Operational,$.75


#### Merge the two dataframes

In [8]:
# Merge the two DataFrames (data_df and meters_df) together based on the Meter Ids they share
merge_df = pd.merge(data_df, meters_df, on= "meter_id")
merge_df.dropna(subset=['meter_id'])
merge_df

,ticket_id,issue_date,fine,violation_code,entity,car_make,issue_time,latitude,longitude,total_tickets,officer_badge,amount_paid,location,meter_id,meter_status,rate
0,5565039450,2015205,$30.00,29,3487595,DODG,2035,0,0,0,17,$30.00,GREENWOOD LOT,F48,Operational,NaN
1,5561331314,2013043,$25.00,29,3284270,CHEV,2025,0,0,0,19,$25.00,GREENWOOD LOT,F48,Operational,NaN
2,5564857413,2015178,$30.00,29,3485309,FORD,2151,0,0,0,5,$45.00,GREENWOOD LOT,F48,Operational,NaN
3,5564881165,2015154,$30.00,29,3142735,TOYT,2125,0,0,0,20,$65.00,GREENWOOD LOT,F48,Operational,NaN
4,5563439740,2014223,$25.00,29,3409265,DODG,2021,0,0,0,19,$25.00,GREENWOOD LOT,F48,Operational,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414494,5568030653,2017341,$30.00,29,3610782,PLYM,939,39956022,83000591,1,2,$0.00,MAIN ST W@14TH AVE E,WE52,Mtr/Post Remv - PERMANENT,NaN
414495,5562719436,2014053,$25.00,29,3369240,CHRY,836,0,0,0,7,$40.00,5TH ST S,SQ421,Operational,$.75
414496,5563229154,2014175,$25.00,29,3398117,CHEV,837,0,0,0,16,$60.00,BROAD ST W,WN2265,Mtr/Post Remv - PERMANENT,NaN
414497,5561967533,2013239,$25.00,29,3104911,SATU,1124,0,0,0,2,$25.00,LONG ST E,ER609,Mtr/Post Remv - PERMANENT,NaN


#### Clean the merge data further

 - Drop latitude/longitude columns with values of 0
 - Add decimal points to both latitude and longitude
 - Set the the values in longitude column to all negative.
 - Convert Julian dates to normal dates in a dataframe and delete the old index
 - For loop to convert the military time into regular time and save it into list
 - Convert object to float
 - Reset index and delete old index

In [9]:
# Drop latitude/longitude columns with values of 0:
merge_df.drop(merge_df.index[merge_df['latitude'] == 0], inplace = True)
merge_df.drop(merge_df.index[merge_df['longitude'] == 0], inplace = True)

# Add decimal points to both latitude and longitude
merge_df['latitude'] = merge_df['latitude'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
merge_df['longitude'] = merge_df['longitude'].apply(lambda x: x / 10 ** (len((str(x))) - 2))

# Set the longitude column to be all negative. 
merge_df.longitude = merge_df.longitude*(-1) 

# Convert Julian dates to normal dates in a dataframe 
merge_df['date_issued'] = (pd.to_datetime((merge_df.issue_date // 1000).astype(str)) + 
                 pd.to_timedelta(merge_df.issue_date % 1000, unit='D'))

# Delete the old column with julian date from which it was converted from
merge_df = merge_df.loc[:, ~merge_df.columns.str.contains('^issue_date')]

# For loop to convert the military time into regular time and save it into list
time_list = []
for time in merge_df['issue_time']:
    x = datetime.strptime(str(time),'%H%M').strftime('%I:%M %p')
    time_list.append(x)

merge_df['issue_time'] = time_list

# Reset index and delete old index
merge_df.reset_index(drop=True)

C:\Users\Saiteja\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ticket_id,fine,violation_code,entity,car_make,issue_time,latitude,longitude,total_tickets,officer_badge,amount_paid,location,meter_id,meter_status,rate,date_issued
0,5568069816,$30.00,29,3654282,HYUN,09:21 PM,39.985932,-83.006150,0,18,$30.00,W/S GREENWOOD LOT,F48,Operational,NaN,2017-12-21
1,5568044664,$30.00,29,3669089,GMC,08:08 PM,39.985960,-83.006220,0,18,$0.00,W/S GREENWOOD LOT,F48,Operational,NaN,2017-12-13
2,5568330382,$30.00,29,2802650,NISS,08:34 AM,39.985906,-83.006300,0,8,$30.00,GREENWOOD LOT@HIGH ST N,F48,Operational,NaN,2018-03-31
3,5568407732,$30.00,29,3163426,SAAB,09:25 PM,39.985811,-83.006048,0,19,$0.00,GREENWOOD LOT@HIGH ST N,F48,Operational,NaN,2018-04-14
4,5568241110,$30.00,29,3681300,CHEV,09:16 PM,39.985901,-83.006216,0,18,$0.00,W/S GREENWOOD LOT,F48,Operational,NaN,2018-03-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19721,5568136946,$30.00,29,3674707,SUBA,02:08 PM,39.955167,-82.992405,0,15,$30.00,FIFTH ST S@MOUND ST E,SQ611,Operational,$.40,2018-01-27
19722,5568326145,$50.00,72,3685744,HOND,01:26 PM,39.958784,-83.010578,0,16,$0.00,STATE ST W@STARLING ST S,C911,Operational,$.40,2018-03-30
19723,5568025064,$55.00,14,3667807,HOND,04:28 PM,39.956205,-82.999667,0,13,$0.00,E S HIGH ST S,SK151,Operational,$.75,2017-12-06
19724,5568363691,$30.00,29,3687660,TOYT,12:45 PM,39.966242,-82.992826,0,15,$0.00,6TH ST N@LONG ST E,NS106,Operational,$.40,2018-04-07


In [10]:
# Drop a row observation by condition
merge_df= merge_df[merge_df.amount_paid != '($50.00)']

In [11]:
# Convert object to float
merge_df['fine'] = merge_df['fine'].str.replace('$', '').astype(float)
merge_df['amount_paid']= merge_df['amount_paid'].str.replace('$', '').astype(float)

In [12]:
merge_df.dtypes

ticket_id                 object
fine                     float64
violation_code             int64
entity                     int64
car_make                  object
issue_time                object
latitude                 float64
longitude                float64
total_tickets              int64
officer_badge             object
amount_paid              float64
location                  object
meter_id                  object
meter_status              object
rate                      object
date_issued       datetime64[ns]
dtype: object

In [13]:
# Reset index and delete old index
merge_df.reset_index(drop=True)

,ticket_id,fine,violation_code,entity,car_make,issue_time,latitude,longitude,total_tickets,officer_badge,amount_paid,location,meter_id,meter_status,rate,date_issued
0,5568069816,30.0,29,3654282,HYUN,09:21 PM,39.985932,-83.006150,0,18,30.0,W/S GREENWOOD LOT,F48,Operational,NaN,2017-12-21
1,5568044664,30.0,29,3669089,GMC,08:08 PM,39.985960,-83.006220,0,18,0.0,W/S GREENWOOD LOT,F48,Operational,NaN,2017-12-13
2,5568330382,30.0,29,2802650,NISS,08:34 AM,39.985906,-83.006300,0,8,30.0,GREENWOOD LOT@HIGH ST N,F48,Operational,NaN,2018-03-31
3,5568407732,30.0,29,3163426,SAAB,09:25 PM,39.985811,-83.006048,0,19,0.0,GREENWOOD LOT@HIGH ST N,F48,Operational,NaN,2018-04-14
4,5568241110,30.0,29,3681300,CHEV,09:16 PM,39.985901,-83.006216,0,18,0.0,W/S GREENWOOD LOT,F48,Operational,NaN,2018-03-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19720,5568136946,30.0,29,3674707,SUBA,02:08 PM,39.955167,-82.992405,0,15,30.0,FIFTH ST S@MOUND ST E,SQ611,Operational,$.40,2018-01-27
19721,5568326145,50.0,72,3685744,HOND,01:26 PM,39.958784,-83.010578,0,16,0.0,STATE ST W@STARLING ST S,C911,Operational,$.40,2018-03-30
19722,5568025064,55.0,14,3667807,HOND,04:28 PM,39.956205,-82.999667,0,13,0.0,E S HIGH ST S,SK151,Operational,$.75,2017-12-06
19723,5568363691,30.0,29,3687660,TOYT,12:45 PM,39.966242,-82.992826,0,15,0.0,6TH ST N@LONG ST E,NS106,Operational,$.40,2018-04-07


In [14]:
#Saving the final Dataframe to CSV
merge_df.to_csv('final_data.csv', encoding='utf-8', index=False)

In [15]:
#Generate descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values.
merge_df['fine'].describe()

count    19725.000000
mean        38.977896
std         39.207132
min         22.000000
25%         30.000000
50%         30.000000
75%         50.000000
max        500.000000
Name: fine, dtype: float64

In [16]:
# Confirming our primary key is unique
merge_df['ticket_id'].nunique()

19725

# Connect to local database

In [17]:
rds_connection_string = "postgres:postgres@localhost:5432/parking_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

#### Create new data (parking_fines) with select columns 

In [18]:
parking_fines_df = merge_df[['ticket_id', 'fine', 'amount_paid', 'date_issued', 'issue_time']].copy()
parking_fines_df.reset_index(drop=True)

,ticket_id,fine,amount_paid,date_issued,issue_time
0,5568069816,30.0,30.0,2017-12-21,09:21 PM
1,5568044664,30.0,0.0,2017-12-13,08:08 PM
2,5568330382,30.0,30.0,2018-03-31,08:34 AM
3,5568407732,30.0,0.0,2018-04-14,09:25 PM
4,5568241110,30.0,0.0,2018-03-04,09:16 PM
...,...,...,...,...,...
19720,5568136946,30.0,30.0,2018-01-27,02:08 PM
19721,5568326145,50.0,0.0,2018-03-30,01:26 PM
19722,5568025064,55.0,0.0,2017-12-06,04:28 PM
19723,5568363691,30.0,0.0,2018-04-07,12:45 PM


#### Create new data (parking_cars) with select columns

In [25]:
parking_cars_df = merge_df[['ticket_id', 'entity', 'car_make', 'total_tickets']].copy()
parking_cars_df.reset_index(drop=True)

,ticket_id,entity,car_make,total_tickets
0,5568069816,3654282,HYUN,0
1,5568044664,3669089,GMC,0
2,5568330382,2802650,NISS,0
3,5568407732,3163426,SAAB,0
4,5568241110,3681300,CHEV,0
...,...,...,...,...
19720,5568136946,3674707,SUBA,0
19721,5568326145,3685744,HOND,0
19722,5568025064,3667807,HOND,0
19723,5568363691,3687660,TOYT,0


#### Create new data (parking_location) with select columns

In [20]:
parking_locations_df = merge_df[['ticket_id', 'latitude', 'longitude']].copy()
parking_locations_df.reset_index(drop=True)

,ticket_id,latitude,longitude
0,5568069816,39.985932,-83.006150
1,5568044664,39.985960,-83.006220
2,5568330382,39.985906,-83.006300
3,5568407732,39.985811,-83.006048
4,5568241110,39.985901,-83.006216
...,...,...,...
19720,5568136946,39.955167,-82.992405
19721,5568326145,39.958784,-83.010578
19722,5568025064,39.956205,-82.999667
19723,5568363691,39.966242,-82.992826


#### Make sure the above three tables are created in postgres as well before you start the next steps

### Check for tables

In [21]:
engine.table_names()

['parking_cars', 'parking_fines', 'parking_location']

### Use pandas to load csv converted DataFrame into database

In [22]:
parking_fines_df.to_sql(name='parking_fines', con=engine, if_exists='append', index=False)

In [26]:
parking_cars_df.to_sql(name='parking_cars', con=engine, if_exists='append', index=False)

In [24]:
parking_locations_df.to_sql(name='parking_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [ ]:
pd.read_sql_query('select * from parking_fines', con=engine).head()

In [ ]:
pd.read_sql_query('select * from parking_cars', con=engine).head()

In [ ]:
pd.read_sql_query('select * from parking_location', con=engine).head()